In [ ]:
!pip install datasets transformers scikit-learn evaluate emoji

In [ ]:
!pip install emoji

In [ ]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import sklearn
import numpy as np
import evaluate

In [ ]:
from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [ ]:
DATA_PATH = '/content/drive/MyDrive/LMs-gmail/'
DATA_FILE = 'tweetsentbr.csv'
MODEL_NAME = 'melll-uff/bertweetbr'
OUTPUT_PATH = '/content/drive/MyDrive/LMs-gmail/'

In [ ]:
train_file = DATA_PATH + 'tweetsentbr_' + 'train.csv'
test_file = DATA_PATH +  'tweetsentbr_' + 'test.csv'

dataset = load_dataset('csv', data_files={'train': train_file, 'test': test_file})

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset = dataset.rename_column("tweet", "text")


In [ ]:
dataset = dataset.rename_column("class", "label")


In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 6215
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1554
    })
})

In [ ]:
from emoji import demojize

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, do_lower_case=False, normalization=True)

tokenizer.demojizer = lambda x: demojize(x, language='pt')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/799k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/17.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
def preprocess_function(examples):
  return tokenizer(examples["text"], truncation=True)


tokenized = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/6215 [00:00<?, ? examples/s]

Map:   0%|          | 0/1554 [00:00<?, ? examples/s]

In [ ]:
tokenized

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 6215
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1554
    })
})

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
# Carregar métricas
#accuracy = evaluate.load("accuracy")
#f1 = evaluate.load("f1")
#recall = evaluate.load("recall")
#precision = evaluate.load("precision")

def compute_metrics(eval_pred):

    #predictions, labels = eval_pred
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    print(predictions, labels)

    #predictions = np.argmax(predictions, axis=1)
    clf_metrics = evaluate.combine(["accuracy", "f1", "precision", "recall"])
    return clf_metrics.compute(predictions=predictions, references=labels)

    #return {
     #   "accuracy": accuracy.compute(predictions=predictions, references=labels),
      #  "f1": f1.compute(predictions=predictions, references=labels, average='macro'),
      #  "recall": recall.compute(predictions=predictions, references=labels, average='macro'),
       # "precision": precision.compute(predictions=predictions, references=labels, average='macro')
    #}


In [ ]:
#id2label = {0: "negativo", 1: "positivo"}

#label2id = {"negativo": 0, "positivo": 2}

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME, num_labels=2 #, id2label=id2label, label2id=label2id
)


config.json:   0%|          | 0.00/801 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/540M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at melll-uff/bertweetbr and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir=OUTPUT_PATH,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="epoch",
    do_eval=True,
    save_strategy="epoch",
    save_total_limit=2
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 